In [2]:
import re
import pprint as pp

In [12]:
file_in = 'Zscaler Cloud Portal - Firewall Control.html'
#file_in = 'Zscaler Cloud Portal - Print All Policies.html'
#file_in = 'Zscaler Cloud Portal - URL & Cloud App Control.html'
#file_in = 'Zscaler Cloud Portal - SSL Inspection.html'
file_out = file_in.split('.')[0] + '.csv'
#file_out = 'rules_table.txt'

In [13]:
def getDescriptions(data):
    regex_descr = '<div class="policy-table-criteria-item-label"><span class="policy-table-cell-text -js-table-cell-text">(.*?)</span></div>'
    result = re.findall(regex_descr, data)
    for i in result:
        print(i)

In [14]:
def getContent(data):
    regex_content = r"<span class=\"policy-table-criteria-item-data-item ?\">(.*?)<\/span>"
    result = re.findall(regex_content, data)
    return(result)

In [6]:
def getRuleName(data):
    regex_text = r"<span class=\"policy-table-cell-text -js-table-cell-text ?\">(.*?)</span>"
    #regex_text = r"<span class=\"policy-table-cell-text -js-table-cell-text\">(.*?)</span>"
    result = re.search(regex_text, data)
    return(result[1])

In [7]:
def getActionContent(data):
    regex_action = r"<span class=\"policy-table-action-content\">(.*?)</span>"
    result = re.search(regex_action, data)
    return(result[1])

In [8]:
def getTagLocations(data, tag):
    locationsList = []
    result_iter = re.finditer(tag, data)
    for result in result_iter:      
        locationsList.append((result.start(), result.end()))
    return locationsList

In [9]:
def getContentBetweenTags(data, tag):
    intervalData = []
    locations = getTagLocations(data, tag)
    end_prev = 0
    start_prev = 0
    for interval in locations:
        start = interval[0]
        end = interval[1]
        if end_prev == 0:
            end_prev = end
            start_prev = start
        else:
            #Here I should put function which parses all fields
            #-------------------------------------------------
            intervalData.append(data[start_prev:start])
            #-------------------------------------------------
            end_prev = end
            start_prev = start
    #row is needed to add match of last find pattern to the end of file
    intervalData.append(data[start_prev:len(data)])
    return intervalData

In [10]:
tag0 = r'<span class=\"policy-table-rule-name -js-table-cell policy-table-cell\" data-section=\"name\" style=\"width: calc\(\d+.?\d+% - \d+.?\d+px\); height: \d+px;\">'
tag1 = r'<div class=\"policy-table-criteria-item-label\"><span class=\"policy-table-cell-text -js-table-cell-text\">'


In [17]:
with open(file_in, 'r') as file:
    data = file.read()

with open(file_out, 'w', encoding='utf8') as output:
    content_array = getContentBetweenTags(data, tag0)
    #this block iterates through whole row content
    for table_rows in content_array:
        rule_name = getRuleName(table_rows)
        action_content = getActionContent(table_rows)
        
        #this block splits row into subcategories and iterates
        for table_items in getContentBetweenTags(table_rows, tag1):
            group_name = getRuleName(table_items)
            table_values = getContent(table_items)
            for value in table_values:
                csv_row = rule_name + '|' + group_name + '|' + value + '|' + action_content
                print(csv_row)
                output.write(csv_row + '\n')

Malicious URLs Block|Destination Addresses|185.220.101.154|Block/Drop
Malicious URLs Block|Destination Addresses|188.165.162.233|Block/Drop
Malicious URLs Block|Destination Addresses|18.66.2.96|Block/Drop
Malicious URLs Block|Destination Addresses|52.31.66.13|Block/Drop
Malicious URLs Block|Destination Addresses|87.98.235.184|Block/Drop
Malicious URLs Block|Destination Addresses|92.222.139.190|Block/Drop
Malicious URLs Block|Destination Addresses|185.204.219.141|Block/Drop
Malicious URLs Block|Destination Addresses|86.111.241.251|Block/Drop
Malicious URLs Block|Destination Addresses|216.21.13.14|Block/Drop
Malicious URLs Block|Destination Addresses|13.226.158.17|Block/Drop
Malicious URLs Block|Destination Addresses|212.180.238.58|Block/Drop
Malicious URLs Block|Destination Addresses|94.152.162.245|Block/Drop
Malicious URLs Block|Destination Addresses|88.85.94.236|Block/Drop
Malicious URLs Block|Destination Addresses|94.237.39.40|Block/Drop
Malicious URLs Block|Destination Addresses|139